In [0]:
import pandas as pd
import requests
import json

In [0]:
url= "https://data.wa.gov/api/odata/v4/769e-73q6"

data = []

while url:
    response = requests.get(url)
    response.raise_for_status()
    json_data = response.json()
    pd_temp = pd.json_normalize(json_data.get('value'))
    data.append(pd_temp)
    url = json_data.get('@odata.nextLink')

In [0]:
dl_df = pd.concat(data)

drop_cols=['country_of_origin', 'iso_2_alpha_code','iso_3_alpha_code','iso_numeric_code','irs_country_code','geocoded_column.type']

dl_df = dl_df.drop(columns=drop_cols,axis= 1)

dl_df.rename(columns={'count_of_new_credential': 'cred_count'}, inplace=True)

spark_df = spark.createDataFrame(dl_df) 

spark_df.write.format("delta")\
        .mode("overwrite")\
        .saveAsTable("main.wadl.dl_transfer")

In [0]:
%sql
SELECT
year,
state_of_origin,
sum(cred_count) as total_cred
FROM main.wadl.dl_transfer GROUP BY year, state_of_origin

Databricks visualization. Run in Databricks to view.